In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt

In [2]:
learning_rate = 0.01
number_digits = 2

In [ ]:
Train_data = pd.read_csv(r"C:\Users\91999\Downloads\mnist_train.csv\mnist_train.csv")
Test_data = pd.read_csv(r'C:\Users\91999\Desktop\Project\mnist_test.csv')

Train_data = Train_data.loc[((Train_data['label'] == 0) | (Train_data['label'] == 1))]
Test_data = Test_data.loc[((Test_data['label'] == 0) | (Test_data['label'] == 1))]

In [ ]:
Train_data = np.array(Train_data)
Train_data = Train_data.transpose()

Test_data = np.array(Test_data)
Test_data = Test_data.transpose()

Train_tensor = torch.tensor(Train_data)
Test_tensor = torch.tensor(Test_data)

X_Train = np.float32(Train_data[1:])
Y_Train = Train_data[0]

X_Test = np.float32(Test_data[1:])
Y_Test = Test_data[0]

X_Train = X_Train / 255
X_Test = X_Test / 255

In [5]:
input_dim,input_num = X_Train.shape
print(input_dim,input_num)

hidden_layer = 300
output_layer = number_digits

784 12665


In [6]:
#Here inp_dim = number of features for each input (in this case 28 x 28 = 784) and m = number of inputs (in this case 60000)

def init_params():    #np.random.rand initializes uniformly from (-0.5, 0.5).
    W1 = np.random.rand(hidden_layer,input_dim) - 0.5
    b1 = np.random.rand(hidden_layer,1) - 0.5
    W2 = np.random.rand(number_digits,hidden_layer) - 0.5
    b2 = np.random.rand(number_digits,1)  - 0.5
    theta = np.random.rand(1,input_dim) - 0.5

    return W1, b1, W2, b2, theta


def ReLU(Z):
    return np.maximum(0,Z)

def softmax(Z):
    max_element = np.amax(Z)
    sum = 0.0
    for i in range(len(Z)):
        sum = sum + np.exp(Z[i] - max_element)
    c = np.exp(Z - max_element)
    A = c / sum
    return A


def ReLU(Z):
    return np.maximum(0,Z)

def softmax(Z):
    max_element = np.amax(Z)
    sum = 0.0
    for i in range(len(Z)):
        sum = sum + np.exp(Z[i] - max_element)
    c = np.exp(Z - max_element)
    A = c / sum
    return A


def d_ReLU(Z):
    return Z>0

def convert_digit_positional_representation(Y):
    size = Y.size
    positional_representation = np.zeros((size, number_digits))    #positional_representation is a size x number_digits matrix filled with zeros.
    positional_representation[np.arange(size),Y] = 1    #Now if i'th entry of Y is 3 then the i'th row of positional_representation will be 0001000000
    positional_representation = positional_representation.T
    return positional_representation

In [7]:
def forward_prop(W1, b1, W2, b2, theta, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    Z2 = Z2 + theta.dot(X)  #theta.dot(X) is a scalar and Z2 is a vector. This equation is adding that scalar to every component of the Z2 vector. 
    A2 = softmax(Z2)        #A2 is a 10 x 1 column vector with positive entries which correspond to probabilities with sum = 1 and one entry is >> others.
    return Z1, A1, Z2, A2

def backward_prop(Z1,A1,Z2,A2,W1,W2,b2,X,Y):     ## loss = 1/m * sum(y_pred - y)^2
    positional_representation = convert_digit_positional_representation(Y)
    dZ2 = A2 - positional_representation 
    dW2 = dZ2.dot(A1.T) #10 x 1 times 1 x 300 = 10 x 300
    db2 = dZ2 #10 x 1 times 1 x 1 = 10 x 1
    dZ1 = W2.T.dot(dZ2) * d_ReLU(Z1) 
    dW1 = dZ1.dot(X.T) 
    db1 = dZ1
    
    return dW1, db1, dW2, db2



def update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,alfa):
    W1 = W1 - alfa * dW1
    b1 = b1 - alfa * db1
    W2 = W2 - alfa * dW2
    b2 = b2 - alfa * db2
    return W1,b1,W2,b2


def gradient_descent(X,Y,alfa,epochs):
    W1,b1,W2,b2,theta = init_params()
    for i in range(epochs):
        for j in range(np.shape(X)[1]):
            X_n=X[:, j:j+1:]
            Y_n = Y[j]
            Z1,A1,Z2,A2 = forward_prop(W1,b1,W2,b2,theta,X_n)
            dW1,db1,dW2,db2 = backward_prop(Z1,A1,Z2,A2,W1,W2,b2,X_n,Y_n)
            W1,b1,W2,b2 = update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,alfa)

    return W1, b1, W2, b2, theta

In [8]:
epochs = 1

W1, b1, W2, b2, theta = gradient_descent(X_Train,Y_Train,learning_rate,epochs)

print("Gradient descent is over")

Gradient descent is over


In [9]:
def get_pred(A2):    #Returns the index of the maximum elements of the array A2
    return np.argmax(A2,0)

def get_accuracy(pred , Y):
    counter = 0
    
    #print(pred , Y)
    for j in range(len(pred)):
        if(pred[j] - Y[j] < 0.001 and pred[j] - Y[j] > -0.001):
            counter  = counter+1
    return counter/Y.size

def make_pred(X,W1,b1,W2,b2,theta):
    pred = []
    for j in range(np.shape(X)[1]):
        X_n=X[:, j:j+1:]
        _,_,_,A2 = forward_prop(W1,b1,W2,b2,theta,X_n)
        pred.append(get_pred(A2))
    return pred


pred = make_pred(X_Test,W1,b1,W2,b2,theta)
pred_accuracy = get_accuracy(pred,Y_Test)
print(pred_accuracy)

0.9985815602836879


In [10]:
#### Lasso starting 

def l1_min_norm_numpy_array(W_array):
    rows, col = W_array.shape
    min = float('inf')
    for i in range(rows):
        for j in range(col):
            new_min= abs(W_array[i,j])
            if(new_min < min):
                min = new_min

    return min


def l1_max_norm_numpy_array(W_array):
    rows, col = W_array.shape
    max = float('-inf')
    for i in range(rows):
        for j in range(col):
            new_max= abs(W_array[i,j])
            if(new_max > max):
                max = new_max

    return max


def S_lambda(x,lambda_const):
    return np.sign(x) * max(abs(x) - lambda_const, 0)

def Print_unique_m(H, w_m):
    H_row = len(H)
    print(H_row)
    print("Printing the unique m")
    for m in range(H_row+1):
        if(m == 0 and w_m[m] >= H[m]):
            m_t = 0
            print(0, w_m[0], H[0])
        if(m > 0 and m < H_row and H[m] <= w_m[m] and w_m[m] <= H[m-1]):
            m_t = m
            print(m, H[m], w_m[m], H[m-1])

        if(m == H_row and w_m[m] <= H[m-1]):  #To handle the case of m = 299
            m_t = H_row
            print(H_row, H[H_row-1], w_m[m])

    print("Finished printing the unique m")


In [16]:
def Hier_Prox(theta,W1,alpha_lambda,M,set_zero_theta_j):
    W1_row, W1_col = W1.shape
    for j in range(W1_col):
        if(theta[0,j] == 0):
            continue
        H = []
        #sorting the entries.
        for i in range(W1_row):
           H.append(abs(W1[i,j]))
        H.sort(reverse=True)

        sum = 0
        w_m = []
        for m in range(W1_row+1):     
            w_m.append((M/(1+m*(M*M))) * S_lambda((abs(theta[0,j]) + M * sum), alpha_lambda))
            if(m < W1_row):
               sum = sum + H[m]       
       
        m_t = -1
        #Finding small m_t s.t. W1[m+1,j]<= w_m <= W1[m,j]
        #Print_unique_m(H, w_m)
        for m in range(W1_row+1):
            if(m == 0 and w_m[m] >= H[m]):
                m_t = 0
                break
            elif(m > 0 and m < W1_row and H[m] <= w_m[m] and w_m[m] <= H[m-1]):
                m_t = m
                break
            elif(m == W1_row and w_m[m] <= H[m-1]):  #To handle the case of m = 299
                m_t = W1_row
        
      
        theta[0,j] = (1/M) * np.sign(theta[0,j]) * w_m[m_t]
        if(theta[0,j] == 0):
            set_zero_theta_j.add(j)
            for i in range(W1_row):
                 W1[i,j] = 0
        else:
            for i in range(W1_row):
                 W1[i,j] = np.sign(W1[i,j]) * min(w_m[m_t],abs(W1[i,j]))

            
    return theta,W1,set_zero_theta_j


In [17]:
def Lasso(X,Y,W1,b1,W2,b2,theta,epochs, M,epsilon,alfa,p_lambda):
    W1_row, W1_col = W1.shape
    #Step 3 - Initialize the penalty, p_lambda and the number of active features, k = d
    k = input_dim
    set_zero_theta_j = set()
    #Step 4 - while k>0
    while(k>0):
        p_lambda = (1+epsilon)*p_lambda
        for i in range (epochs):
            #Step 2 - training the feed forward neural network on L(theta,W)
            for j in range(np.shape(X)[1]):
                X_n=X[:, j:j+1:]
                Y_n = Y[j]
                Z1,A1,Z2,A2 = forward_prop(W1,b1,W2,b2,theta,X_n)
                dW1, db1, dW2, db2 = backward_prop(Z1,A1,Z2,A2,W1,W2,b2,X_n,Y_n)
                theta_update_factor = db2[Y_n]
                dtheta = theta_update_factor * X_n   # which is same as dZ2.dot(X_n.T)  
                W1,b1,W2,b2 = update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,alfa)
                theta  = theta - alfa * dtheta.T
              
            if(len(set_zero_theta_j) > 0):
                for index_k in set_zero_theta_j:
                    theta[0,index_k] = 0
                    for i in range(W1_row):
                        W1[i,index_k] = 0
            theta,W1,set_zero_theta_j = Hier_Prox(theta,W1,p_lambda,M,set_zero_theta_j)
            
            print(theta.shape)
            print(l1_min_norm_numpy_array(theta))

         
        #updating value of k to be the non-zero coordinate of theta.
        temp_k = np.count_nonzero(theta)
        if(k > temp_k):
            k = temp_k
            print("The value of k is:",k)
        
        
        pred = make_pred(X_Test,W1,b1,W2,b2,theta)
        pred_accuracy = get_accuracy(pred,Y_Test)
        print(pred_accuracy)
        
    return W1,b1,W2,b2,theta

In [ ]:
#Default values used in that paper: hierarchy multiplier M = 10,  learning_rate = .01, early stopping criterion = 10, 
#penalty lambda = 5,1, or 0.1, epsilon (path multiplier) = 0.01

p_lambda = 1
epsilon = 0.001
M = 10
epochs = 10

#W1,b1,W2,b2,theta = Lasso(X_Train,Y_Train,W1,b1,W2,b2,theta,1,30,0.01,0.01)
W1,b1,W2,b2,theta = Lasso(X_Train,Y_Train,W1,b1,W2,b2,theta,epochs,M,epsilon,learning_rate,p_lambda)


(1, 784)
0.04715035194257549
(1, 784)
0.0459555498560257
(1, 784)
0.045202711671666874
(1, 784)
0.04464731573453448
(1, 784)
0.0441495628090286
(1, 784)
0.04368754738380634
(1, 784)
0.04325756314821175
(1, 784)
0.04285717807930461
(1, 784)
0.042479781086733
(1, 784)
0.04213989157977836
0.9995271867612293
(1, 784)
0.041805935324711656
(1, 784)
0.04148373462184759
(1, 784)
0.0411752899105321
(1, 784)
0.040876391216315595
(1, 784)
0.04058172382265554
(1, 784)
0.04030110642039269
(1, 784)
0.04002589743233988
(1, 784)
0.03976286736138851
(1, 784)
0.03952288123786326
(1, 784)
0.03929968514374744
0.9995271867612293
(1, 784)
0.03907916625691617
(1, 784)
0.038862015422997255
(1, 784)
0.038648598430148756
(1, 784)
0.03843773531998433
(1, 784)
0.038233060867384205
(1, 784)
0.038028400464860175
(1, 784)
0.037827816380171624
(1, 784)
0.037630491246974505
(1, 784)
0.03744328615163928
(1, 784)
0.037268357106950624
0.9995271867612293
(1, 784)
0.037095869020316884
(1, 784)
0.036928882904010404
(1, 784)

In [ ]:
The value of k is: 26
0.9990543735224586

The value of k is: 24
0.9952718676122931

The value of k is: 23
0.9938534278959811

The value of k is: 22
0.9881796690307328

The value of k is: 21
0.9777777777777777

The value of k is: 20
0.9768321513002364

The value of k is: 19
0.9645390070921985

The value of k is: 18
0.9470449172576832

The value of k is: 17
0.9385342789598109

The value of k is: 16
0.8936170212765957

The value of k is: 15
0.8794326241134752

The value of k is: 14
0.8600472813238771

The value of k is: 13
0.8037825059101655

The value of k is: 12
0.7957446808510639

The value of k is: 11
0.7513002364066194

The value of k is: 10
0.7432624113475177

The value of k is: 9
0.7205673758865249

The value of k is: 8
0.6950354609929078

The value of k is: 7
0.6789598108747045

The value of k is: 6
0.5910165484633569